In [4]:
from pathlib import Path
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy
from tqdm import tqdm

## Define brain, find trace data folder

In [8]:
#specify brain1 or brain2 below
brain = "brain1"

root_dir = Path(os.path.abspath('')).parents[1]
experiment_dir = os.path.join(root_dir, "axon_geometry")
data_dir = os.path.join(experiment_dir, "data", brain)
segments_swc_dir = os.path.join(data_dir, "segments_swc")
trace_data_dir = os.path.join(data_dir, "trace_data")
trace_data_dir = os.path.join(trace_data_dir, "14")
print(f"Directory where swcs reside: {segments_swc_dir}")

Directory where swcs reside: /Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/axon_geometry/data/brain1/segments_swc


## Read trace data

In [9]:
max_id = 300


def classify_height(row):
    height = row["height"]
    if height <= 2:
        return height
    else:
        return 3

def numerical_class(row):
    _class = row["class"]
    if _class == "axon":
        return 0
    if _class == "collateral":
        return 1
    if _class == "terminal":
        return 2

df_path = os.path.join(trace_data_dir, "df.csv")
if os.path.exists(df_path):
    df = pd.read_csv(df_path)
else:
    df = pd.DataFrame(columns=["seg_id", "class", "height", "log_seg_length", "measure", "value", "log_value"])
    for i in np.arange(0, max_id):
        i = int(i)
        trace_data_path = os.path.join(trace_data_dir, "{}.npy".format(i))
        if os.path.exists(trace_data_path) is True:
            trace_data = np.load(trace_data_path, allow_pickle=True)
            print("Loaded segment {}".format(i))

            for node in trace_data:
                seg_length = node["seg_length"]
                height = node["height"]
                _class = node["class"]
                mean_curvature = node["mean_curvature"]
                mean_torsion = node["mean_torsion"]
                
                log_seg_length = np.log10(seg_length)

                log_mean_curvature = np.log10(mean_curvature)
                df = df.append({"seg_id": i, "height": height, "class": _class, "log_seg_length": log_seg_length, "measure": "curvature", "value": mean_curvature, "log_value": log_mean_curvature}, ignore_index=True)

                log_mean_torsion = np.log10(mean_torsion)
                df = df.append({"seg_id": i, "height": height, "class": _class, "log_seg_length": log_seg_length, "measure": "torsion", "value": mean_torsion, "log_value": log_mean_torsion}, ignore_index=True)
    df.to_csv(df_path)
df["class"] = df.apply(numerical_class, axis=1)
df["height_class"] = df.apply(classify_height, axis=1)

# Wilcoxon test
Assume samples from same neuron are independent


In [10]:
n_perms = 10000

measures = ["curvature", "torsion"]

classes = {0: "primary", 1: "collateral", 2: "terminal"}
n = len(classes)
matrix_pairs = np.triu(np.ones((n, n)), k=1)
(coord_pairs_x, coord_pairs_y) = np.where(matrix_pairs == 1)

for measure in measures:
    for class_1, class_2 in zip(coord_pairs_x, coord_pairs_y):
        Y1 = []
        Y2 = []
        segment_numbers = []
        X_class = []
        neuron_id = 0
        #collect data from all neurons
        for i in np.arange(0, max_id):
            sample_query = df.loc[(df['seg_id'] == i) & ((df['class'] == class_1)) & (df['measure'] == measure)]
            num_segments = len(sample_query.index)
            if num_segments > 0:
                Y1.append(np.mean(sample_query["value"].to_numpy()))
                sample_query = df.loc[(df['seg_id'] == i) & ((df['class'] == class_2)) & (df['measure'] == measure)]
                Y2.append(np.mean(sample_query["value"].to_numpy()))
            
        _,p = scipy.stats.wilcoxon(Y1, Y2)

        if p < 0.05/6:
            if np.mean(Y1) > np.mean(Y2):
                symb = ">"
            else:
                symb = "<"
            print(f"{classes[class_1]} {symb} {classes[class_2]} in {measure}")
            print(f"p-val was: {p}")
        else:
            print(f"No difference between {classes[class_1]} and {classes[class_2]} in {measure}")
            print(f"p-val was: {p}")



primary < collateral in curvature
p-val was: 4.017296952216419e-31
primary < terminal in curvature
p-val was: 4.672925996383199e-29
collateral > terminal in curvature
p-val was: 7.574172613730026e-06
primary < collateral in torsion
p-val was: 9.825129434514524e-25
primary > terminal in torsion
p-val was: 1.370123591903152e-11
collateral > terminal in torsion
p-val was: 1.0682346584784447e-23
